# Распознавание рукописных цифр

Воспользуемся выборкой из уже подготовленной БД изображений:

MNIST – (Modified National Institute of Standards and Technology)

60 000 изображений в обучающей выборке и 10 000 – в тестовой.
Каждое изображение имеет размер 28х28 пикселей и представлено в градациях серого, т.е. каждый пиксел имеет значение от 0 до 255 (0 – черный цвет, 255 – белый):

* x_train – изображения цифр обучающей выборки;
* y_train – вектор соответствующих значений цифр (например, если на i-м изображении нарисована 5, то  y_train[i] = 5);
* x_test – изображения цифр тестовой выборки;
* y_test – вектор соответствующих значений цифр для тестовой выборки.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# отображение первых 25 изображений из обучающей выборки
plt.figure(figsize=(10,5))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
 
plt.show()

Соберем обычную полносвязную ИНС с
* 28 x 28 = 784 входами;
* 128 нейронами скрытого слоя + bias;
* 10 нейронами выходного слоя + bias.

In [ ]:
model = keras.Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
 
print(model.summary())     # вывод структуры НС в консоль

### Cтандартизация входных данных
каждое значение тензоров x_train и x_test будет делиться на максимальное число 255, которое они могут принимать. На выходе получим вещественные величины от 0 до 1.

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

На выходе ожидаем вектор с 1 на месте соответствующего числа, т.к. наша НС имеет 10 выходов, и каждый выход будет соответствовать определенной цифре: от 0 до 9.

Воспользуемся стандартной функцией Keras:

In [ ]:
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

Для задачи классификации стоит выбрать функцию потерь категориальной кросс-энтропии - categorical_crossentropy.

Построить алгоритм минимизации процента ошибок распознавания цифр сложно (если вообще возможно), поэтому мы привязываемся к более простому с точки зрения математики критерию – категориальной кросс-энтропии (S), минимизируя которую, надеемся, что будет уменьшаться и процент ошибок. 
$${Q(S(y), L)=-\sum_i(L_i*log(S_i))}$$



In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Обучим модель
* batch_size = 32 – это размер батча (32 картинки), после которых будет выполняться корректировка весов;
* validation_split = 0,2 – разбиение обучающей выборки на собственно обучающую и проверочную. Значение 0,2 определяет, что для каждой эпохи 20% случайных картинок из обучающей выборки будут помещаться в выборку валидации.

In [ ]:
model.fit(x_train, y_train_cat, batch_size=32, epochs=10, validation_split=0.2)

Проверка

In [ ]:
model.evaluate(x_test, y_test_cat)

Контроль по примеру

In [ ]:
n = 3
plt.imshow(x_test[n], cmap=plt.cm.binary)
plt.show()

In [ ]:
x = np.expand_dims(x_test[n], axis=0)
res = model.predict(x)
print( res )

In [ ]:
print( np.argmax(res) )

Пропустим через НС всю тестовую выборку и векторы выходных значений преобразуем в числа от 0 до 9:

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred, axis=1)
 
print(pred.shape)
 
print(pred[:20])
print(y_test[:20])

# Все ли картинки распознаются верно?

Затем, сформируем маску, которая будет содержать True для верных вариантов и False – для неверных. И с помощью этой маски выделим из тестовой выборки все неверные результаты:

In [ ]:
mask = pred == y_test
print(mask[:10])
 
x_false = x_test[~mask]
y_false = x_test[~mask]
 
print(x_false.shape)

И выведем первые 5 из них на экран:

In [ ]:
for i in range(5):
  print("Значение сети: "+str(y_test[i]))
  plt.imshow(x_false[i], cmap=plt.cm.binary)
  plt.show()